In [1]:
import yfinance as yf
# import argparse
import atexit
import datetime
import logging
import schedule
import time
import json

In [2]:
logger_format = '%(asctime)-15s %(message)s'
logging.basicConfig(format=logger_format)
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [3]:
def log_exception(msg,exception):
  logger.warning(f'{msg}, caused by, {exception}')

In [4]:
from kafka import KafkaProducer
from kafka.errors import KafkaError, KafkaTimeoutError

In [5]:
topic_name = 'stockTopic'
kafka_broker = '127.0.0.1:9092'

In [6]:
def fetch_symbol_price(symbol,producer):
  '''
  function to fetch price of the stock and push it to the producer instance
  of kafka
  :param symbol: symbol of the stock
  :param producer: instance of the kafka producer
  :return: None
  '''

  logger.info(f'Fetching stock price of {symbol}')
  try:
#     print(yf.Ticker(symbol).info)
    price=yf.Ticker(symbol).info['ask']
    timestamp=datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%dT%H:%M:%SZ')
    payload={
        'StockSymbol':symbol,
        'Price':price,
        'Time':timestamp
    }
    producer.send(topic=topic_name, value=payload)
    logger.info(f'Received stock price of {symbol} at {timestamp} : {price}')
  except KafkaError as ke:
    logger.warning(f'Kafka Error {ke}')
  except Exception as e:
    logger.warning(f'Failed to fetch stock price of {symbol} caused by {e}')
  return payload

In [7]:
def on_shutdown(producer):
  '''
  shutdown hook to be called before the shutdown
  :param producer: instance of a kafka producer
  :return: None
  '''
  try:
    logger.info('Flushing pending messages to kafka, timeout is set to 10s')
    producer.flush(10) 
    logger.info('Finish flushing pending messages to kafka')
  finally:
      try:
        logger.info('Closing the Kafka connection')
        producer.close(10)
      except Exception as e:
        log_exception('Failed to close kafka connection',e)

In [8]:
if __name__=='__main__':
    symbol='GC=F'
    producer=KafkaProducer(
        bootstrap_servers=kafka_broker,
        value_serializer=lambda x: json.dumps(x).encode('utf-8')
    )
    schedule.every(1).second.do(fetch_symbol_price,symbol,producer)
    atexit.register(on_shutdown,producer)
    while True:
        schedule.run_pending()
        time.sleep(1)

2021-11-26 22:21:06,440 Fetching stock price of GC=F
2021-11-26 22:21:11,986 Received stock price of GC=F at 2021-11-26T22:21:11Z : 1791.3
2021-11-26 22:21:12,999 Fetching stock price of GC=F
2021-11-26 22:21:18,615 Received stock price of GC=F at 2021-11-26T22:21:18Z : 1791.3
2021-11-26 22:21:19,620 Fetching stock price of GC=F
2021-11-26 22:21:24,759 Received stock price of GC=F at 2021-11-26T22:21:24Z : 1791.3
2021-11-26 22:21:25,774 Fetching stock price of GC=F
2021-11-26 22:21:31,148 Received stock price of GC=F at 2021-11-26T22:21:31Z : 1791.3
2021-11-26 22:21:32,158 Fetching stock price of GC=F
2021-11-26 22:21:37,129 Received stock price of GC=F at 2021-11-26T22:21:37Z : 1791.3
2021-11-26 22:21:38,136 Fetching stock price of GC=F
2021-11-26 22:21:43,851 Received stock price of GC=F at 2021-11-26T22:21:43Z : 1791.3
2021-11-26 22:21:44,862 Fetching stock price of GC=F
2021-11-26 22:21:50,016 Received stock price of GC=F at 2021-11-26T22:21:50Z : 1791.3
2021-11-26 22:21:51,023 Fet

2021-11-26 22:28:31,925 Fetching stock price of GC=F
2021-11-26 22:28:45,208 Received stock price of GC=F at 2021-11-26T22:28:45Z : 1783.5
2021-11-26 22:28:46,221 Fetching stock price of GC=F
2021-11-26 22:29:06,848 Received stock price of GC=F at 2021-11-26T22:29:06Z : 1861.3
2021-11-26 22:29:07,855 Fetching stock price of GC=F
2021-11-26 22:29:23,824 Received stock price of GC=F at 2021-11-26T22:29:23Z : 1861.3
2021-11-26 22:29:24,832 Fetching stock price of GC=F
2021-11-26 22:29:40,747 Received stock price of GC=F at 2021-11-26T22:29:40Z : 1861.3
2021-11-26 22:29:41,753 Fetching stock price of GC=F
2021-11-26 22:29:54,448 Received stock price of GC=F at 2021-11-26T22:29:54Z : 1861.3
2021-11-26 22:29:55,456 Fetching stock price of GC=F
2021-11-26 22:30:09,781 Received stock price of GC=F at 2021-11-26T22:30:09Z : 1861.3
2021-11-26 22:30:10,792 Fetching stock price of GC=F
2021-11-26 22:30:23,141 Received stock price of GC=F at 2021-11-26T22:30:23Z : 1861.3
2021-11-26 22:30:24,154 Fet

KeyboardInterrupt: 